In [4]:
from localtileserver import TileClient, get_leaflet_tile_layer
import leafmap.foliumap as leafmap

import pystac
import json
import numpy as np
import os

os.environ["GTIFF_SRS_SOURCE"] = "EPSG"

Load the JSON result listing generated by `cwltool`:

In [5]:
with open("/workspace/workshop/07_app_package/water-bodies/outputs/out.json") as f: 
    results = json.load(f)

Look for the `catalog.json` file:

In [6]:
for item in results["stac_catalog"]["listing"]:
    
    if item['basename'] == "catalog.json":
        catalog = pystac.read_file(item["path"])
        break
        

List the contents of the STAC Catalog

In [7]:
catalog.describe()

* <Catalog id=catalog>
  * <Item id=S2B_10TFK_20210713_0_L2A>
  * <Item id=S2A_10TFK_20220524_0_L2A>


In [8]:
it1 = catalog.get_item('S2B_10TFK_20210713_0_L2A')
it1

id: S2B_10TFK_20210713_0_L2A
"bbox: [-121.413752588606, 39.83402935827303, -120.71922542174708, 40.47202226335379]"
proj:epsg: 32610
"proj:geometry: {'type': 'Polygon', 'coordinates': [[[635710.0, 4411780.0], [693380.0, 4411780.0], [693380.0, 4481380.0], [635710.0, 4481380.0], [635710.0, 4411780.0]]]}"
"proj:bbox: [635710.0, 4411780.0, 693380.0, 4481380.0]"
"proj:shape: [6960, 5767]"
"proj:transform: [10.0, 0.0, 635710.0, 0.0, -10.0, 4481380.0, 0.0, 0.0, 1.0]"
"proj:projjson: {'$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json', 'type': 'ProjectedCRS', 'name': 'WGS 84 / UTM zone 10N', 'base_crs': {'name': 'WGS 84', 'datum': {'type': 'GeodeticReferenceFrame', 'name': 'World Geodetic System 1984', 'ellipsoid': {'name': 'WGS 84', 'semi_major_axis': 6378137, 'inverse_flattening': 298.257223563}}, 'coordinate_system': {'subtype': 'ellipsoidal', 'axis': [{'name': 'Geodetic latitude', 'abbreviation': 'Lat', 'direction': 'north', 'unit': 'degree'}, {'name': 'Geodetic longitude', 'abbreviation': 'Lon', 'direction': 'east', 'unit': 'degree'}]}, 'id': {'authority': 'EPSG', 'code': 4326}}, 'conversion': {'name': 'UTM zone 10N', 'method': {'name': 'Transverse Mercator', 'id': {'authority': 'EPSG', 'code': 9807}}, 'parameters': [{'name': 'Latitude of natural origin', 'value': 0, 'unit': 'degree', 'id': {'authority': 'EPSG', 'code': 8801}}, {'name': 'Longitude of natural origin', 'value': -123, 'unit': 'degree', 'id': {'authority': 'EPSG', 'code': 8802}}, {'name': 'Scale factor at natural origin', 'value': 0.9996, 'unit': 'unity', 'id': {'authority': 'EPSG', 'code': 8805}}, {'name': 'False easting', 'value': 500000, 'unit': 'metre', 'id': {'authority': 'EPSG', 'code': 8806}}, {'name': 'False northing', 'value': 0, 'unit': 'metre', 'id': {'authority': 'EPSG', 'code': 8807}}]}, 'coordinate_system': {'subtype': 'Cartesian', 'axis': [{'name': 'Easting', 'abbreviation': '', 'direction': 'east', 'unit': 'metre'}, {'name': 'Northing', 'abbreviation': '', 'direction': 'north', 'unit': 'metre'}]}, 'id': {'authority': 'EPSG', 'code': 32610}}"
datetime: 2021-07-13T19:03:24Z
https://stac-extensions.github.io/projection/v1.0.0/schema.json
href: ./otsu.tif


In [9]:
it1.properties

{'proj:epsg': 32610,
 'proj:geometry': {'type': 'Polygon',
  'coordinates': [[[635710.0, 4411780.0],
    [693380.0, 4411780.0],
    [693380.0, 4481380.0],
    [635710.0, 4481380.0],
    [635710.0, 4411780.0]]]},
 'proj:bbox': [635710.0, 4411780.0, 693380.0, 4481380.0],
 'proj:shape': [6960, 5767],
 'proj:transform': [10.0, 0.0, 635710.0, 0.0, -10.0, 4481380.0, 0.0, 0.0, 1.0],
 'proj:projjson': {'$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json',
  'type': 'ProjectedCRS',
  'name': 'WGS 84 / UTM zone 10N',
  'base_crs': {'name': 'WGS 84',
   'datum': {'type': 'GeodeticReferenceFrame',
    'name': 'World Geodetic System 1984',
    'ellipsoid': {'name': 'WGS 84',
     'semi_major_axis': 6378137,
     'inverse_flattening': 298.257223563}},
   'coordinate_system': {'subtype': 'ellipsoidal',
    'axis': [{'name': 'Geodetic latitude',
      'abbreviation': 'Lat',
      'direction': 'north',
      'unit': 'degree'},
     {'name': 'Geodetic longitude',
      'abbreviation': 'Lon',


In [10]:
def get_center_from_bbox(bbox):
    x1, y1, x2, y2 = bbox
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    center = (center_y, center_x) # use (lat, lon) format convention
    return center

In [11]:
center = get_center_from_bbox(it1.bbox)
print(center)

(40.15302581081341, -121.06648900517655)


In [41]:
m = leafmap.Map(center=center, zoom = 9)

for item in catalog.get_all_items():
    m.add_raster(item.get_assets()["data"].get_absolute_href(), layer_name=item.id)#, crs="EPSG:32610")

m

Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed


HTTPError: 400 Client Error: BAD REQUEST for url: http://127.0.0.1:37859/api/bounds?units=EPSG:4326&projection=EPSG:3857&filename=%2Fworkspace%2Fworkshop%2F07_app_package%2Fwater-bodies%2Foutputs%2FS2B_10TFK_20210713_0_L2A%2Fotsu.tif

In [33]:
for item in catalog.get_all_items():
    print(item)

<Item id=S2B_10TFK_20210713_0_L2A>
<Item id=S2A_10TFK_20220524_0_L2A>
